In [1]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_ollama import ChatOllama

In [2]:
# import pandas as pd 

# chunk_metadata = pd.read_csv('chunks.csv')
# chunk_metadata['text'][561]

In [3]:
import os
# Force GPU usage
os.environ["OLLAMA_ACCELERATOR"] = "gpu"

chat_model = ChatOllama(
    model="gemma3:1b",
    num_predict = 512,
    temperature=0,
    top_k=10,
    additional_kwargs = {'gpu': True})

In [4]:
label_template = """
Bạn là trợ lý AI được giao nhiệm vụ tạo ra một cặp câu hỏi-trả lời thực tế duy nhất dựa trên một vài tài liệu nhất định về pháp luật Việt Nam, về luật an ninh mạng và an toàn thông tin mạng, các sản phẩm về công nghệ thông tin, mật mã, dữ liệu người dùng. 
Câu hỏi phải là thứ mà người dùng có thể tự nhiên hỏi khi tìm kiếm thông tin có trong tài liệu.
Với tài liệu: {chunk}

Hãy tuân theo các yêu cầu sau khi tạo câu hỏi - câu trả lời:

1. Phân tích các chủ đề, sự kiện và quy định quan trọng trong tài liệu đã cho, chọn một chủ đề cụ thể liên quan đến luật an ninh mạng hoặc an toàn thông tin mạng.
2. Tạo ra một câu hỏi tự nhiên mà người dùng có thể hỏi để tìm kiếm thông tin trong tài liệu này.
3. Sử dụng ngôn ngữ tự nhiên, có thể bao gồm lỗi đánh máy nhỏ hoặc cách diễn đạt thông thường của người dùng khi tìm kiếm.
4. Đảm bảo câu hỏi có liên quan về mặt ngữ nghĩa với nội dung tài liệu MÀ KHÔNG sao chép trực tiếp các cụm từ.
5. Đảm bảo rằng tất cả các câu hỏi đều tập trung vào một chủ đề tương tự hoặc yêu cầu cùng một dạng thông tin.
6. Đảm bảo câu hỏi và câu trả lời được viết bằng tiếng Việt.
7: Đảm bảo rằng đầu ra phải là một JSON hợp lệ chứa cả 'question' và 'answer'. Nếu đầu ra không chứa cả 'question' và 'answer', kết quả sẽ bị coi là không hợp lệ.

Trả về một đối tượng JSON có cấu trúc như sau:
{{
  "question": "Văn bản câu hỏi đã tạo",
  "answer": "Văn bản câu trả lời đã tạo"
}}

LƯU Ý QUAN TRỌNG: Chỉ trả về đối tượng JSON, không thêm bất kỳ văn bản nào khác. Không thêm ghi chú, giải thích, hoặc bất kỳ thông tin nào ngoài JSON.
"""

label_prompt = ChatPromptTemplate.from_template(label_template, stream = False)

label_chain = label_prompt | chat_model | JsonOutputParser()
# label_chain = label_prompt | chat_model

In [5]:
import pandas as pd
from tqdm.auto import tqdm
import random

# Define the number of pairs per chunk
min_pairs = 2
max_pairs = 3

max_try = 5

# Initialize an empty list to store the pairs
pairs = []

error_chunks = []

chunk_metadata = pd.read_csv('chunks.csv')
# Iterate over the chunks
for i in tqdm(range(len(chunk_metadata))):
    # Invoke the chain to generate the pairs
    chunk = chunk_metadata['text'][i]
    cid = chunk_metadata['cid'][i]
    num_pairs = random.randint(min_pairs, max_pairs)

    for _ in range(num_pairs):
        for _ in range(max_try):
            try:
                result = label_chain.invoke({"chunk": chunk})
                # Extract the question and answer from the result
                question = result['question']
                answer = result['answer']
                # Create a dictionary with the question, answer, and corpus id
                pair = {'question': question, 'answer': answer, 'cid': cid, 'corpus': chunk}
                # Append the pair to the list
                pairs.append(pair)
                break
            except Exception as e:
                print(f"Error: {e}")
        else:
            error_chunks.append(cid)

# Create a pandas DataFrame from the list of pairs
df = pd.DataFrame(pairs)
df.to_csv('train_data.csv', index=False)

/home/quangminh/anaconda3/envs/chatbots/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  6%|▋         | 37/590 [02:43<40:35,  4.41s/it]

Error: Invalid json output: ```json
{
 "question": "Luật an ninh quốc gia, trật tự, an toàn xã hội bao gồm các hành vi nào liên quan đến việc vi phạm thông tin trên không gian mạng?",
 "answer": "Các hành vi liên quan đến việc vi phạm pháp luật về an ninh quốc gia, trật tự, an toàn xã hội bao gồm đăng tải, phát tán thông tin trên không gian mạng có nội dung quy định tại các khoản 1, 2, 3, 4 và 5 Điều 16 và hành vi quy định tại khoản 1 Điều 17 của Luật này; chiếm đoạt tài sản; tổ chức đánh bạc, đánh bạc qua mạng Internet; trộm cước cước viễn thông quốc tế trên nền Internet; vi phạm bản quyền và sở hữu trí tuệ trên không gian mạng; giả mạo trang thông tin điện tử của cơ quan, tổ chức, cá nhân; làm giả, lưu hành, trộm cắp, mua bán, thu thập, trao đổi trái phép thông tin thẻ tín dụng, tài khoản ngân hàng của người khác; phát hành, cung cấp, sử dụng trái phép các phương tiện thanh toán; hướng dẫn người khác thực hiện hành vi vi phạm pháp luật;".
}
```
For troubleshooting, visit: https://pyt

  6%|▋         | 38/590 [03:18<2:06:42, 13.77s/it]

Error: Invalid json output: ```json
{
 "question": "Luật an ninh quốc gia, trật tự, an toàn xã hội bao gồm các hành vi nào liên quan đến việc vi phạm thông tin trên không gian mạng?",
 "answer": "Các hành vi liên quan đến việc vi phạm pháp luật về an ninh quốc gia, trật tự, an toàn xã hội bao gồm đăng tải, phát tán thông tin trên không gian mạng có nội dung quy định tại các khoản 1, 2, 3, 4 và 5 Điều 16 và hành vi quy định tại khoản 1 Điều 17 của Luật này; chiếm đoạt tài sản; tổ chức đánh bạc, đánh bạc qua mạng Internet; trộm cước cước viễn thông quốc tế trên nền Internet; vi phạm bản quyền và sở hữu trí tuệ trên không gian mạng; giả mạo trang thông tin điện tử của cơ quan, tổ chức, cá nhân; làm giả, lưu hành, trộm cắp, mua bán, thu thập, trao đổi trái phép thông tin thẻ tín dụng, tài khoản ngân hàng của người khác; phát hành, cung cấp, sử dụng trái phép các phương tiện thanh toán; hướng dẫn người khác thực hiện hành vi vi phạm pháp luật;".
}
```
For troubleshooting, visit: https://pyt

 48%|████▊     | 284/590 [21:04<27:53,  5.47s/it] 

...

 98%|█████████▊| 581/590 [45:02<00:57,  6.39s/it]

Error: Invalid json output: ```json
{
 "question": "Luật bảo vệ dữ liệu người dùng trong lĩnh vực ứng dụng di động và các dịch vụ trực tuyến có những quy định cụ thể nào?",
 "answer": "Luật bảo vệ dữ liệu người dùng trong lĩnh vực ứng dụng di động và các dịch vụ trực tuyến được quy định tại Nghị định 13/2023/ND-CP về bảo vệ dữ liệu cá nhân, có các quy định quan trọng như: quy định về việc thu thập, sử dụng, lưu trữ, xử lý và bảo vệ dữ liệu cá nhân, đảm bảo dữ liệu được bảo mật, an toàn và không được sử dụng cho mục đích khác ngoài mục đích đã nêu.  Nghị định này cũng quy định về quyền của người dùng, quyền truy cập, sửa đổi, yêu cầu xóa dữ liệu, và trách nhiệm của các tổ chức, cá nhân trong việc bảo vệ dữ liệu.  Ngoài ra, có quy định về việc xử lý dữ liệu theo nguyên tắc "chỉ lấy, chỉ sử dụng, chỉ lưu trữ dữ liệu cần thiết" và có quy định về việc bảo vệ dữ liệu trong quá trình truyền tải, lưu trữ và xử lý dữ liệu."
}
```
For troubleshooting, visit: https://python.langchain.com/docs/tro

 99%|█████████▊| 582/590 [45:27<01:35, 11.88s/it]

Error: Invalid json output: ```json
{
 "question": "Luật bảo vệ dữ liệu người dùng trong lĩnh vực ứng dụng di động và các dịch vụ trực tuyến có những quy định cụ thể nào?",
 "answer": "Luật bảo vệ dữ liệu người dùng trong lĩnh vực ứng dụng di động và các dịch vụ trực tuyến được quy định tại Nghị định 13/2023/ND-CP về bảo vệ dữ liệu cá nhân, có các quy định quan trọng như: quy định về việc thu thập, sử dụng, lưu trữ, xử lý và bảo vệ dữ liệu cá nhân, đảm bảo dữ liệu được bảo mật, an toàn và không được sử dụng cho mục đích khác ngoài mục đích đã nêu.  Nghị định này cũng quy định về quyền của người dùng, quyền truy cập, sửa đổi, yêu cầu xóa dữ liệu, và trách nhiệm của các tổ chức, cá nhân trong việc bảo vệ dữ liệu.  Ngoài ra, có quy định về việc xử lý dữ liệu theo nguyên tắc "chỉ lấy, chỉ sử dụng, chỉ lưu trữ dữ liệu cần thiết" và có quy định về việc bảo vệ dữ liệu trong quá trình truyền tải, lưu trữ và xử lý dữ liệu."
}
```
For troubleshooting, visit: https://python.langchain.com/docs/tro

100%|██████████| 590/590 [46:54<00:00,  4.77s/it]


In [6]:
print(len(error_chunks))

3


In [10]:
len(df)

1442